# Harmonic oscillator

Numerical solution of simple harmonic oscillator, and double spring oscillator.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
from dataclasses import dataclass
import os

plt.style.use("bmh")
#plt.style.use("dark_background")

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100
%config InlineBackend.figure_format = 'retina'
%config InteractiveShellApp.matplotlib = "inline"


In [ ]:
print("Making sure current working directory is in this directory")
try:
    os.chdir("../..")
    pass
except Exception as e:
    print("Current directory:", os.getcwd())
    print(e)
    print("New directory", os.getcwd())

In [ ]:
!pwd
%load_ext autoreload
%autoreload 2

# from ode.harmonic_oscillator1.solvers import \

from solvers import \
    SolverResult, \
    BackwardEulerSolver, \
    ForwardEulerSolver, \
    SemiImplicitEulerSolver1, \
    SemiImplicitEulerSolver2, \
    SemiImplicitEulerSolverAvg, \
    VodeSolver, \
    VelocityVerletSolver

from equations import \
    DoubleSpring1, \
    HarmonicOscillator1

# Simple 1D spring without dampening

```
/|
/|--vvvv----- m
/|
 ----------> x axis
```

- k = spring_constant
- L = resting_length
- F = ma

$$ F = -k(x - L) = ma $$

$$=>   m dv/dt = -k(x - L)$$

### System of equations:

$$
\begin{align}
dx/dt &= v                 =   0        +    1 * v \\
dv/dt &= -k/m (x - L)      = -k/m x     +    0 * v    + kL/m
\end{align}
$$

Substitute $x = x- L$ to remove the constant term:

$$
\begin{align}
dx/dt &= v \\
dv/dt &= -k/m x
\end{align}
$$

Matrix form: Let the vector $\vec x$ contain the entire state of the system. 

$$\vec x = \begin{bmatrix}x\\ v\end{bmatrix}$$

The equation above becomes the following:

$$\frac{d\vec x}{dt} = \begin{bmatrix}0 & 1\\ -k/m & 0\end{bmatrix} \, \vec x = f(t, x)$$

Some methods additionally require a Jacobian matrix, so we compute that as well:

$$
J = J(t, x) = \frac{df}{dx}= \begin{bmatrix}0 & 1\\ -k/m & 0\end{bmatrix}
$$

Furthermore, symplectic integrators require splitting the state vector $\mathbf x$ into generalized position and momentum coordinates:

$$\mathbf x = \begin{bmatrix}\mathbf q\\ \mathbf p\end{bmatrix},$$

and splitting $\mathbf f$ into $f_q(t, q)$

$$\mathbf f(\mathbf t, \mathbf q, \mathbf p) = \begin{bmatrix}\mathbf f_q(t, p)\\ \mathbf f_p(t, q)\end{bmatrix},$$

## Solver comparison functions

In [ ]:
def plot_solver_results(solver, dts, r0, t_start=0.0, t_end=0.05):
    compute_times = []
    #plt.figure(figsize=(0.4,0.2))
    for dt in dts:
        tt = np.arange(t_start, t_end, dt)
        result = solver.solve(tt, r0)
        compute_times.append(result.compute_time)
        positions = result.xs[0,:]
        plt.plot(tt*1000, positions*100)
        plt.title(f"Simple harmonic oscillator: {solver.name}")

    legends = [f"dt={dt}, compute={compute:.2f}" for (dt, compute) in zip(dts, compute_times)]
    plt.legend(legends)
    plt.xlabel("milliseconds")
    plt.ylabel("cm displacement")
    plt.show()

def plot_solver_comparison(equation_name, solvers, t_start=0.0, t_end=0.074):
    plt.figure(figsize=(16, 6))
    compute_times = []
    for (solver, dt, r0) in solvers:
        tt = np.arange(t_start, t_end, dt)
        result = solver.solve(tt, r0)
        compute_times.append(result.compute_time)
        positions = result.xs[0,:]
        times = result.ts
        plt.plot(times*1000, positions*100)

    plt.title(f"{equation_name}")
    legends = [f"{solver.name}, dt={dt:.2e}, compute={compute:.2f}" for ((solver, dt), compute) in zip(solvers, compute_times)]
    plt.legend(legends)
    plt.xlabel("milliseconds")
    plt.ylabel("cm displacement")
    plt.show()



## The unreasonable accuracy of VODE

VODE is just extremely good even for large time steps.

In [ ]:



equation = HarmonicOscillator1()
f, J, fq, fp = equation.f, equation.J, equation.fq, equation.fp
r0 = np.array([1.0, 0.0])
q0 = r0[0]
p0 = r0[1]

dts = [0.0005, 0.0025]
plot_solver_results(ForwardEulerSolver(f), dts, r0)
plot_solver_results(BackwardEulerSolver(f, J), dts, r0)
plot_solver_results(SemiImplicitEulerSolver1(fq, fp), dts, r0)
plot_solver_results(VodeSolver(f, J), dts, r0)

# Double springs

```

/|     k1      m1       k2       m2
/|---vvvvv----###----vvvvvv-----###
/|
```

In [ ]:
def plot_double_spring_results(solver, dt, r0=None, q0=None, p0=None, t_start=0.0, t_end=0.025):
    t_start = 0.0
    compute_times = []
    tt = np.arange(t_start, t_end, dt)
    if r0 is not None:
        result = solver.solve(tt, r0)
    else:
        result = solver.solve(tt, q0, p0)
    positions_mass0 = result.xs[0,:]
    positions_mass1 = result.xs[1,:]
    #plt.plot(tt*1000, (positions_mass0)*100)
    plt.plot(tt*1000, (positions_mass0)*100)
    plt.plot(tt*1000, (positions_mass1)*100)
    plt.title(f"Double spring: {solver.name}")

    legends = ["Mass 1", "Mass 2"]
    plt.legend(legends)
    plt.xlabel("milliseconds")
    plt.ylabel("cm displacement")
    plt.show()

equation = DoubleSpring1(k0=5000.0, m0=0.2, k1=4000.0, m1=0.1, L0=0.1, L1=0.2)

f, J, fq, fp = equation.f, equation.J, equation.fq, equation.fp
dt = 0.0001

r0 = np.array([0.06, 0.21, 0.0, 0.0])
q0 = r0[:2]
p0 = r0[2:]
plot_double_spring_results(VodeSolver(f, J), dt, r0=r0)
plot_double_spring_results(VelocityVerletSolver(fq, fp), dt, q0=q0, p0=p0)

plot_double_spring_results(ForwardEulerSolver(f), dt, r0=r0)
plot_double_spring_results(SemiImplicitEulerSolver2(fq, fp), dt, q0=q0, p0=p0)
#plot_double_spring_results(BackwardEulerSolver(f, J), dts, r0)
#plot_double_spring_results(VodeSolver(f, J), dt, r0=r0)

# Comparing the accuracy vs VODE

Plotting function

In [ ]:
def plot_double_spring_solver_comparison(results: SolverResult):
    plt.figure(figsize=(14,6))
    for result in results:
        plt.plot(result.ts, result.xs[0,:], linewidth=1)
    plt.legend([result.solver_name for result in results])
    plt.ylabel('displacement')
    plt.xlabel('time')


Define the equation and initial conditions

In [ ]:
f, J, fq, fp = equation.f, equation.J, equation.fq, equation.fp

r0 = np.array([0.06, 0.21, 0.0, 0.0])
q0 = r0[:2]
p0 = r0[2:]

Solve the equation using multiple solvers and compare the results

In [ ]:
dt_precise = 0.00001
tt_precise = np.arange(0, 0.2, dt)
exact_solution = VodeSolver(f, J).solve(tt_precise, r0)

## Velocity verlet solver for double spring

In [ ]:
dt = 0.001
tt = np.arange(0, 0.2, dt)
tt_precise = np.arange(0, 0.2, dt)
plot_double_spring_solver_comparison([
    VelocityVerletSolver(fq, fp).solve(tt, q0, p0),
    exact_solution,
])

## Explicit euler for double spring

In [ ]:
dt = 0.05
tt = np.arange(0, 2.0, dt)
tt_precise = np.arange(0, 2.0, dt)
plot_double_spring_solver_comparison([
    ForwardEulerSolver(f).solve(tt, r0),
    exact_solution,
])

## Plot and animate solutions

First, solve the problem using any solver.

In [ ]:
from solvers import \
    SemiImplicitEulerSolver2, \
    VodeSolver

from equations import \
    DoubleSpring1, \
    HarmonicOscillator1


equation = DoubleSpring1()
f, J, fq, fp = equation.f, equation.J, equation.fq, equation.fp

r0 = np.array([0.05, 0.07, 0.0, 0.0])
q0 = r0[:2]
p0 = r0[2:]
t_end = 2.0

dt_precise = 0.05
tt_precise = np.arange(0, t_end, dt)
exact_solution = VodeSolver(f, J).solve(tt_precise, r0)

dt = 0.05
tt = np.arange(0, t_end, dt)
tt_precise = np.arange(0, t_end, dt)
plot_double_spring_solver_comparison([
    exact_solution,
])

Then, animate the solution using jupycanvas.

In [ ]:
from time import sleep

from ipycanvas import Canvas, hold_canvas
from ipycanvas import Path2D
from math import sin, pi

import scipy.signal

display_sample_rate = 60 # [samples per second]
simulated_time = tt_precise[-1] - tt_precise[0] # [seconds]
animation_frame_count = int(simulated_time * display_sample_rate)

resampled_xs, resampled_ts = scipy.signal.resample(exact_solution.xs, animation_frame_count, exact_solution.ts, axis=1)
animation_dt = resampled_ts[1] - resampled_ts[0]

canvas = Canvas(size=(640,400))
canvas.scale(4000, 4000) # Convert canvas units into meters
display(canvas)

def draw_sissors(x1, x2, wigglyness=5):
    distance = x2 - x1
    s = lambda x: (x - x1)/(x2-x1)
    gimme_y = lambda s: 0.01/2+0.002*sin(s*pi*2*wigglyness)

    canvas.begin_path()
    canvas.move_to(x1, gimme_y(0))
    resolution = 100
    for x in np.linspace(x1, x2, resolution):
        y = gimme_y(s(x))
        canvas.line_to(x, y)
    canvas.line_width = 0.001
    canvas.stroke()

period=0.02
for t, y0, y1 in zip(exact_solution.ts, exact_solution.xs[0,:], exact_solution.xs[1,:]):
    t = float(t)
    y0 = float(y0)
    y1 = float(y1)
    with hold_canvas():
        canvas.clear()
        draw_sissors(0, y0)
        draw_sissors(y0, y1)
        canvas.fill_style = "green"
        canvas.fill_rect(y0-0.01/2, 0, 0.01, 0.01)
        canvas.fill_style = "red"
        canvas.fill_rect(y1-0.01/2, 0, 0.01, 0.01)

    sleep(animation_dt)

# Animating real-time

In [ ]:
from time import sleep
from ipycanvas import Canvas, hold_canvas
from ipycanvas import Path2D
from math import sin, pi, sqrt
import time
import scipy.signal

from equations import \
    DoubleSpring1

m0 = 1
m1 = 2
equation = DoubleSpring1(k0=100.0, m0=m0, k1=200.0, m1=m1, L0=0.05, L1=0.05)
f, J = equation.f, equation.J

frame_rate = 60.0 # Hz
frame_duration = 1.0/frame_rate # s

r0 = np.array([0.05, 0.07, 0.0, 0.0])
t_start = time.monotonic()

simulation = scipy.integrate.BDF(fun=f, t0=t_start, y0=r0, t_bound=1e99, jac=J, max_step=frame_duration)

canvas = Canvas(width=640, height=200)
canvas.scale(4000, 4000) # Convert canvas units into meters
display(canvas)

m0_size, m1_size = sqrt(m0)*0.01, sqrt(m1)*0.01

def draw_sissors(x1, x2, wigglyness=10):
    distance = x2 - x1
    s = lambda x: (x - x1)/(x2-x1)
    gimme_y = lambda s: 0.02+0.002*sin(s*pi*2*wigglyness)

    canvas.begin_path()
    canvas.move_to(x1, gimme_y(0))
    resolution = 20
    for x in np.linspace(x1, x2, resolution):
        y = gimme_y(s(x))
        canvas.line_to(x, y)
    canvas.line_width = 0.0007
    canvas.stroke()

while True:
    actual_t = time.monotonic()
    
    # Step physics forward until they are in front of real time
    num_steps = 0
    while simulation.t < actual_t:
        sim_t_prev, sim_y_prev = simulation.t, simulation.y
        simulation.step()
        num_steps += 1
    #print(num_steps)
    sim_t, sim_y = simulation.t, simulation.y

    #
    #                         a          b
    #                                -

    # Interpolate y between physics frames
    s = (actual_t - sim_t_prev)/(sim_t - sim_t_prev) # From 0 to 1 between sim_t_prev and sim_t
    y = (1 - s)*sim_y_prev + s*sim_y
    
    with hold_canvas():
        canvas.clear()
        draw_sissors(0, y[0])
        draw_sissors(y[0], y[1])
        canvas.fill_style = "green"
        canvas.fill_rect(y[0]-m0_size/2, 0.02-m0_size/2, m0_size, m0_size)
        canvas.fill_style = "red"
        canvas.fill_rect(y[1]-m1_size/2,  0.02-m1_size/2, m1_size, m1_size)
        frame_time_so_far = time.monotonic() - actual_t
        canvas.text_baseline = "top"
        canvas.font = "0.01px Arial";
        canvas.fill_text('%f' % frame_time_so_far, 0.01,0.01),
        if frame_time_so_far > frame_duration:
            canvas.fill_style = "red"
            canvas.fill_rect(0, 0, 1, 1)
    sleep(max(frame_duration - frame_time_so_far, 0))

# Triple spring
Thanks, ChatGPT

In [ ]:
from time import sleep
from ipycanvas import Canvas, hold_canvas
from ipycanvas import Path2D
from math import sin, pi, sqrt
import time
import scipy.signal

from equations import TripleSpring

m0 = 5
m1 = 2
m2 = 5
equation = TripleSpring(k0=100.0, m0=m0, k1=200.0, m1=m1, k2=100.0, m2=m2, L0=0.05, L1=0.05, L2=0.05)
f, J = equation.f, equation.J

frame_rate = 60.0 # Hz
frame_duration = 1.0/frame_rate # s

r0 = np.array([0.05, 0.10, 0.15, 0.1, -0.1, 0.1])
t_start = time.monotonic()

simulation = scipy.integrate.BDF(fun=f, t0=t_start, y0=r0, t_bound=1e99, jac=None, max_step=frame_duration)

canvas = Canvas(width=900, height=200)
canvas.scale(4000, 4000) # Convert canvas units into meters
display(canvas)

m0_size, m1_size, m2_size = sqrt(m0)*0.01, sqrt(m1)*0.01, sqrt(m2)*0.01

def draw_sissors(x1, x2, wigglyness=10):
    distance = x2 - x1
    s = lambda x: (x - x1)/(x2-x1)
    gimme_y = lambda s: 0.02+0.002*sin(s*pi*2*wigglyness)

    canvas.begin_path()
    canvas.move_to(x1, gimme_y(0))
    resolution = 20
    for x in np.linspace(x1, x2, resolution):
        y = gimme_y(s(x))
        canvas.line_to(x, y)
    canvas.line_width = 0.0007
    canvas.stroke()

while True:
    actual_t = time.monotonic()
    
    # Step physics forward until they are in front of real time
    num_steps = 0
    while simulation.t < actual_t:
        sim_t_prev, sim_y_prev = simulation.t, simulation.y
        simulation.step()
        num_steps += 1
    #print(num_steps)
    sim_t, sim_y = simulation.t, simulation.y

    #
    #                         a          b
    #                                -

    # Interpolate y between physics frames
    s = (actual_t - sim_t_prev)/(sim_t - sim_t_prev) # From 0 to 1 between sim_t_prev and sim_t
    y = (1 - s)*sim_y_prev + s*sim_y
    
    with hold_canvas():
        canvas.clear()
        draw_sissors(0, y[0])
        draw_sissors(y[0], y[1])
        draw_sissors(y[1], y[2])
        canvas.fill_style = "green"
        canvas.fill_rect(y[0]-m0_size/2, 0.02-m0_size/2, m0_size, m0_size)
        canvas.fill_style = "red"
        canvas.fill_rect(y[1]-m1_size/2,  0.02-m1_size/2, m1_size, m1_size)
        canvas.fill_style = "blue"
        canvas.fill_rect(y[2]-m2_size/2,  0.02-m2_size/2, m2_size, m2_size)
        frame_time_so_far = time.monotonic() - actual_t
        canvas.text_baseline = "top"
        canvas.font = "0.01px Arial";
        canvas.fill_text('%f' % frame_time_so_far, 0.01,0.01),
        if frame_time_so_far > frame_duration:
            canvas.fill_style = "red"
            canvas.fill_rect(0, 0, 1, 1)
    sleep(max(frame_duration - frame_time_so_far, 0))

# Quad spring

In [ ]:
from time import sleep
from ipycanvas import Canvas, hold_canvas
from ipycanvas import Path2D
from math import sin, pi, sqrt
import time
import scipy.signal

from equations import QuadSpring

# exploding stiffness #rtol=1e-3 or higher.
# m0 = 1
# m1 = 1
# m2 = 1
# m3 = 1
# equation = QuadSpring(k0=100000.0, m0=m0, k1=10000.0, m1=m1, k2=1000.0, m2=m2, k3=100.0, m3=m2, L0=0.05, L1=0.05, L2=0.05, L3=0.05)
# r0 = np.array([0.05, 0.10, 0.15, 0.20, 0.1, 0.1, 0.1, 0.1])

m0 = 4
m1 = 1
m2 = 1
m3 = 4
equation = QuadSpring(k0=1000.0, m0=m0, k1=500.0, m1=m1, k2=100000.0, m2=m2, k3=500.0, m3=m2, L0=0.05, L1=0.05, L2=0.05, L3=0.05)
f, J = equation.f, equation.J

frame_rate = 60.0 # Hz
frame_duration = 1.0/frame_rate # s

r0 = np.array([0.05, 0.10, 0.15, 0.20, 0.2, 0.3, 0.3, 0.0])
slomo = 3
t_start = time.monotonic()/slomo

simulation = scipy.integrate.BDF(fun=f, t0=t_start, y0=r0, t_bound=10e99, jac=J, max_step=frame_duration/2, rtol=1e-5)

canvas = Canvas(width=1000, height=200)
canvas.scale(4000, 4000) # Convert canvas units into meters
display(canvas)

m0_size, m1_size, m2_size, m3_size = sqrt(m0)*0.01, sqrt(m1)*0.01, sqrt(m2)*0.01, sqrt(m3)*0.01

def draw_sissors(x1, x2, wigglyness=10):
    distance = x2 - x1
    s = lambda x: (x - x1)/(x2-x1)
    gimme_y = lambda s: 0.02+0.002*sin(s*pi*2*wigglyness)

    canvas.begin_path()
    canvas.move_to(x1, gimme_y(0))
    resolution = 20
    for x in np.linspace(x1, x2, resolution):
        y = gimme_y(s(x))
        canvas.line_to(x, y)
    canvas.line_width = 0.0007
    canvas.stroke()



while True:
    actual_t = time.monotonic()/slomo
    
    # Step physics forward until they are in front of real time
    num_steps = 0
    while simulation.t < actual_t:
        sim_t_prev, sim_y_prev = simulation.t, simulation.y
        simulation.step()
        num_steps += 1
    #print(num_steps)
    sim_t, sim_y = simulation.t, simulation.y

    # Interpolate y between physics frames
    s = (actual_t - sim_t_prev)/(sim_t - sim_t_prev) # From 0 to 1 between sim_t_prev and sim_t
    y = (1 - s)*sim_y_prev + s*sim_y
    
    with hold_canvas():
        canvas.clear()
        draw_sissors(0, y[0])
        draw_sissors(y[0], y[1])
        draw_sissors(y[1], y[2])
        draw_sissors(y[2], y[3])
        canvas.fill_style = "green"
        canvas.fill_rect(y[0]-m0_size/2, 0.02-m0_size/2, m0_size, m0_size)
        canvas.fill_style = "red"
        canvas.fill_rect(y[1]-m1_size/2,  0.02-m1_size/2, m1_size, m1_size)
        canvas.fill_style = "blue"
        canvas.fill_rect(y[2]-m2_size/2,  0.02-m2_size/2, m2_size, m2_size)
        canvas.fill_style = "magenta"
        canvas.fill_rect(y[3]-m3_size/2,  0.02-m3_size/2, m3_size, m3_size)
        frame_time_so_far = time.monotonic()/slomo - actual_t
        # canvas.text_baseline = "top"
        # canvas.font = "0.01px Arial";
        # canvas.fill_text('%f' % frame_time_so_far, 0.01,0.01),
        # if frame_time_so_far > frame_duration:
        #     canvas.fill_style = "red"
        #     canvas.fill_rect(0, 0, 1, 1)
    sleep(max(frame_duration - frame_time_so_far*slomo, 0))